$p_*$ calculation by [Michael Zingale](https://zingale.github.io/) from [here](https://zingale.github.io/comp_astro_tutorial/07-euler/07-euler-riemann.html)

In [ ]:
import numpy as np
import scipy.optimize as optimize

class State:
    """ a simple object to hold a primitive variable state """

    def __init__(self, p=1.0, u=0.0, rho=1.0):
        self.p = p
        self.u = u
        self.rho = rho

    def __str__(self):
        return f"rho: {self.rho}; u: {self.u}; p: {self.p}"

class RiemannProblem:
    """ a class to define a Riemann problem.  It takes a left
        and right state.  Note: we assume a constant gamma """

    def __init__(self, left_state, right_state, gamma=1.4):
        self.left = left_state
        self.right = right_state
        self.gamma = gamma

        self.ustar = None
        self.pstar = None

    def __str__(self):
        return f"pstar = {self.pstar}, ustar = {self.ustar}"
    
    def u_hugoniot(self, p, side):
        """define the Hugoniot curve, u(p)."""

        if side == "left":
            state = self.left
            s = 1.0
        elif side == "right":
            state = self.right
            s = -1.0

        c = np.sqrt(self.gamma*state.p/state.rho)

        if p < state.p:
            # rarefaction
            u = state.u + s*(2.0*c/(self.gamma-1.0))* \
                (1.0 - (p/state.p)**((self.gamma-1.0)/(2.0*self.gamma)))
        else:
            # shock
            beta = (self.gamma+1.0)/(self.gamma-1.0)
            u = state.u + s*(2.0*c/np.sqrt(2.0*self.gamma*(self.gamma-1.0)))* \
                (1.0 - p/state.p)/np.sqrt(1.0 + beta*p/state.p)

        return u

    def find_star_state(self, p_min=0.001, p_max=1000.0):
        """ root find the Hugoniot curve to find ustar, pstar """

        # we need to root-find on
        self.pstar = optimize.brentq(
            lambda p: self.u_hugoniot(p, "left") - self.u_hugoniot(p, "right"),
            p_min, p_max)
        self.ustar = self.u_hugoniot(self.pstar, "left")

In [ ]:
import csv

def generate_pstar_file(
    path,
    p_min=0.001, p_max=1000, p_bins=7,
    u_min=-1, u_max=1, u_bins=5,
    r_min=0.01, r_max=100, r_bins=5,
    G=5.0/3.0
):
    p_rng = np.logspace(np.log10(p_min), np.log10(p_max), p_bins)
    u_rng = np.linspace(u_min, u_max, u_bins)
    r_rng = np.logspace(np.log10(r_min), np.log10(r_max), r_bins)
    
    fileds = ['rho_r', 'v_r', 'p_r', 'rho_l', 'v_l', 'p_l', 'p_*']

    with open(path, 'w') as csvfile:  
        csvwriter = csv.writer(csvfile, delimiter='\t')
        csvwriter.writerow(fileds)
        
        print('Start...')

        total = (len(p_rng) * len(u_rng) * len(r_rng))**2
        cntr = 0
        percent = 0

        print(f'{percent/10000.0:7.4f}%', end='\r')

        for r_l in r_rng:
            for r_r in r_rng:
                for p_l in p_rng:
                    for p_r in p_rng:
                        for v_l in u_rng:
                            for v_r in u_rng:
                                cntr += 1

                                cs_l = np.sqrt(G * p_l / r_l)
                                cs_r = np.sqrt(G * p_r / r_r)

                                # pressure positivity test
                                if (2. / (G - 1)) * (cs_l + cs_r) <= (v_r - v_l):
                                    continue
                                else:
                                    try:
                                        left = State(p=p_l, u=v_l, rho=r_l)
                                        right = State(p=p_r, u=v_r, rho=r_r)

                                        rp = RiemannProblem(left, right)
                                        rp.find_star_state()
                                        p_star = rp.pstar
                                    except:
                                        continue

                                if p_star is not None:
                                    csvwriter.writerow([r_l, v_l, p_l, r_r, v_r, p_r, p_star])

                                new_percent = int(cntr / total * 1000000)
                                if new_percent > percent:
                                    percent = new_percent
                                    print(f'{percent/10000.0:7.4f}%', end='\r')
    
if __name__ == '__main__' and '__file__' not in globals():
    generate_pstar_file('/Users/saeed/.p_stars/20210619.txt')